This is an [R Markdown](http://rmarkdown.rstudio.com) Notebook. When you execute code within the notebook, the results appear beneath the code. Execute a chunk by clicking the *Run* button within the chunk or by placing your cursor inside it and pressing *Ctrl+Shift+Enter*. 

Add a new chunk by clicking the *Insert Chunk* button on the toolbar or by pressing *Ctrl+Alt+I*.

When you save the notebook, an HTML file containing the code and output will be saved alongside it (click the *Preview* button or press *Ctrl+Shift+K* to preview the HTML file).

The preview shows you a rendered HTML copy of the contents of the editor. Consequently, unlike *Knit*, *Preview* does not run any R code chunks. Instead, the output of the chunk when it was last run in the editor is displayed.

## Objectifs
Le but est d'illustrer les changements dans les distributions d'une série temporelle en fonction du type d'aggrégation spatio-temporelle utilisé.

Pour cela, nous utilisons les simulations pi-control, au pas de temps journalier, du modèle CNRM-CM6-1-HR réalisées lors de l'exercice CMIP6. Nous y avons extrait les variables de température et de précipitation pour la zone géographique suivante: de 0 à 5 degrés de longitude et  de 45.18 à 49.68 degrés de latitude.

Spatialement, nous agrégeons les données pour des patchs de différentes tailles (1x1, 7x5, 11x9 points de grille) centrés autours du point de grille situé à 2.5 degrés de longitude et 47.68 degrés de latitude. L'agrégation est réalisée soit en prenant la moyenne ou le maximum sur l'ensemble des points de grille du patch.

Temporellement, les données sont agrégés soit sur la journée du 15 juillet, soit sur le mois de juillet dans son ensemble, soit sur l'ensemble de l'année. L'agrégation est réalisée soit en prenant la moyenne ou le maximum sur l'ensemble des pas de temps de la période d'intérêt.

Les données sont tout d'abord agrégées spatialement puis temporellement. Ainsi, un type d'agrégation spatio-temporelle donne une série temporelle univariée au pas de temps annuel. Pour un couple d'agrégateurs spatiales (moyenne/maximum) et temporels (moyenne/maximum), les séries temporelles sont stockées sous forme d'une array avec les dimensions suivantes: temps, zone d'agrégation et période d'agrégation.  

Les distributions peuvent être représentées sous forme d'histogrammes (fonction *hist*), de densités estimées par des méthodes à noyau (fonction *density*) ou par des fonctions de répartition empirique (fonction *ecdf*). 

### Chargement des packages


In [ ]:
# A installer avec install.packages("extRemes") 
library(extRemes)



### Lecture des données


In [ ]:
tas_stagg <- readRDS(file = "tas_stagg.rds")
pr_stagg <- readRDS(file = "pr_stagg.rds")
tas <- readRDS(file = "tas.rds")
pr <- readRDS(file = "pr.rds")
dates <- dimnames(tas)[[3]]
str(tas)

tas_smean_tmean <- tas_stagg[[1]][[1]]  
tas_smean_tmax <- tas_stagg[[1]][[2]]  
tas_smax_tmean <- tas_stagg[[2]][[1]]  
tas_smax_tmax <- tas_stagg[[2]][[2]]  
str(tas_smean_tmean)

pr_smean_tmean <- pr_stagg[[1]][[1]]  
pr_smean_tmax <- pr_stagg[[1]][[2]]  
pr_smax_tmean <- pr_stagg[[2]][[1]]  
pr_smax_tmax <- pr_stagg[[2]][[2]]

zones <- dimnames(tas_smax_tmax)[[2]]
periods <- dimnames(tas_smax_tmax)[[3]]
lon <- as.numeric(rownames(tas))
lat <- as.numeric(colnames(tas))



### Tracer la série temporelle des données aggrégées


In [ ]:
# Choisir le type de série temporelle: variable/agrégation spatial/agrégation temporelle
y <- tas_smean_tmean

ylim <- range(y)
par(mfrow = c(3, 3))
for(i in 1:3){
  for(j in 1:3){
    plot(y[, i, j], ylim = ylim, type = "l", main = paste("spatial:", zones[i], "temporal:", periods[j])) 
}}


### Visualisation des distribution

Visualiser le distributions des variables agrégées à l'aide des fonctions R: hist, density et ecdf


In [ ]:
# Histogrammes

breaks <- hist(c(y), breaks = 30, plot = FALSE)$breaks # Définition des classes de valeur.
par(mfrow = c(3, 3))
for(i in 1:3){
  for(j in 1:3){
    hist(y[, i, j], breaks = breaks, freq = FALSE, main = paste("spatial:", zones[i], "temporal:", periods[j])) 
}}


In [ ]:
# Kernel density estimation.

xrange <- range(c(y))
par(mfrow = c(3, 3))
for(i in 1:3){
  for(j in 1:3){

}}


In [ ]:
# Fonction de répartition empirique
xrange <- range(c(y))
par(mfrow = c(3, 3))
for(i in 1:3){
  for(j in 1:3){
   
}}


## Convergence en loi

### Théorème Central Limite.

Si $X_1,X_2,\ldots,X_n$ sont des variables aléatoires réelles indépendantes de même loi de probabilité, d'espérance $\mu$ et de variance $\sigma^2$ alors, lorsque $n$ est suffisamment grand :

la variable aléatoire :
$$ S_n = X_1+X_2+\ldots+X_n$$
suit approximativement une loi normale d'espérance $\mu \times n$ et d'écart-type $\sigma\sqrt n$, notée :
$$ \mathcal N(\mu n,\sigma\sqrt n)$$

source : [wikiversity](https://fr.wikiversity.org/wiki/Th%C3%A9or%C3%A8me_central_limite)


**Exercice :** Sur les graphiques représentant les distributions empiriques, tracer les courbes de densité et de fonction de répartition sous approximation gaussienne. Les paramètres de la loi gaussienne peuvent être estimés par la méthodes des moments, i.e. la moyenne est estimée par la moyenne empirique et la variance par la variance empirique. 

### Théorème de la valeur extrême.

Soit $X_1,X_2\ldots, X_n\ldots$ une séquence de variables indépendantes et identiquement distribuées et $M_n=\max\{X_1,\ldots,X_n\}$. Si une séquence de paires de nombres réels $(a_n, b_n)$ existe telle que $a_n>0$ et
$$\lim_{n \to \infty}P\left(\frac{M_n-b_n}{a_n}\leq x\right) = F(x)$$,
où $F$ est une fonction de distribution non dégénérée, alors la distribution limite $F$ appartient à la famille des lois de Gumbel, lois de Fréchet, ou des lois de Weibull. Ces familles peuvent être regroupées dans la classe des lois d'extremum généralisées de fonction de répartition:

$$F(x;\mu,\sigma,\xi) = \exp\left\{-\left[1+\xi\left(\frac{x-\mu}{\sigma}\right)\right]_+^{-1/\xi}\right\}$$

avec
$\left(1+\xi(x-\mu)/\sigma \right)_+=\max\left( 0 , 1+\xi(x-\mu)/\sigma \right)$
où $\mu\in\mathbb{R}$ est un paramètre de position, $\sigma > 0$ un paramètre de dispersion et $\xi\in\mathbb{R}$ un paramètre de forme appelé ''indice des valeurs extrêmes''.


source : [wikipedia](https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_de_Fisher-Tippett-Gnedenko)


**Exercice :** Sur les graphiques représentant les distributions empiriques, tracer les courbes de densité et de fonction de répartition sous approximation de la loi GEV. Les paramètres de la loi GEV peuvent être estimés via la fonction 'fevd' du package 'extRemes'. La densité et la fonction de répartition de la loi GEV peuvent être respectivement calculées via les fonctions 'devd' et 'pevd' de ce même package.

**Exercice :** Les distributions observées ressemblent-elles aux distributions théoriques ? Sinon pourquoi ?


### Theorie vs Pratique: hypothese i.i.d (independants et identiquement distribués)
L'hypothèse i.i.d est elle réaliste ?

**Exercice :**

- Calculer la moyenne et la variance en chaque point de grille du jeu de donnée initial et les représenter graphiquement avec la fonction *image.plot* du package *fields*.


In [ ]:
# pour la moyenne
map_mean <- 
fields::image.plot(lon, lat, map_mean)

# pour la variance
map_variance <- 
fields::image.plot(lon, lat, map_variance)


- Calculer la matrice des corrélation entre les différents points de grille du jeu de donnée initial et représenter sous forme de carte la corrélation d'un point de grille avec tous les autres.

La matrice de des corrélations peut se calculer avec la fonction *cor*.


In [ ]:
tas_matrix <- matrix(aperm(tas, c(3, 1, 2)), ncol = length(lon) * length(lat))
tas_cor <- 
# Correlation with the first grid point
igridpoint <- 1
fields::image.plot(lon, lat, ))


- Calculer les corrélations temporelles pour chaque point de grille du jeu de donnée initial via la fonction *acf* de R. 



In [ ]:
par(mfrow = c(11, 10))
for(i in 1:11){
  for(j in 1:10){
    plot( , main = paste("spatial:", zones[i], "temporal:", periods[j]))
}}
